In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact
import numpy as np

def visualize_nii(nii_file):
    """
    Visualize slices of a NIfTI (.nii.gz) file in Jupyter Notebook.
    
    Args:
    nii_file (str): Path to the NIfTI file.
    """
    
    # Step 1: Load the NIfTI file using nibabel
    img = nib.load(nii_file)
    
    # Step 2: Get the image data as a numpy array
    img_data = img.get_fdata()

    # Step 3: Determine the number of slices along the z-axis (axial view)
    num_slices = img_data.shape[2]

    # Step 4: Define a function to plot a single slice
    def plot_slice(slice_num):
        plt.figure(figsize=(6, 6))
        plt.imshow(img_data[:, :, slice_num], cmap='gray')
        plt.title(f"Slice {slice_num}")
        plt.axis('off')
        plt.show()

    # Step 5: Use interact to create a slider for browsing through slices
    interact(plot_slice, slice_num=(0, num_slices-1))

# Example Usage: visualize a .nii.gz file
# Replace 'your_nii_file.nii.gz' with the path to your NIfTI file
visualize_nii('/home/local/ASURITE/longchao/Desktop/project/GE_health/CLIP-Driven-Universal-Model/data_temp/single_processed_npy/result/CT_Abd_FLARE22_Tr_0001_img/CT_Abd_FLARE22_Tr_0001_img_Liver.nii.gz')


interactive(children=(IntSlider(value=50, description='slice_num'), Output()), _dom_classes=('widget-interact'…

In [2]:
visualize_nii('/home/local/ASURITE/longchao/Desktop/project/GE_health/SegmentAsYouWish/data/FLARE22Train/npy/CT_Abd/CT_Abd_FLARE22_Tr_0001_img.nii.gz')


interactive(children=(IntSlider(value=50, description='slice_num'), Output()), _dom_classes=('widget-interact'…

In [3]:
visualize_nii("/home/local/ASURITE/longchao/Desktop/project/GE_health/SegmentAsYouWish/data/FLARE22Train/labels/FLARE22_Tr_0001.nii.gz")

interactive(children=(IntSlider(value=54, description='slice_num', max=109), Output()), _dom_classes=('widget-…

In [4]:
visualize_nii("/home/local/ASURITE/longchao/Desktop/project/GE_health/SegmentAsYouWish/data/FLARE22Train/labels/FLARE22_Tr_0001.nii.gz")

interactive(children=(IntSlider(value=54, description='slice_num', max=109), Output()), _dom_classes=('widget-…

In [5]:


def analyze_nii_values(nii_file_path):
    """
    Load a .nii.gz file and analyze the unique values (labels) in the image.
    
    Args:
    nii_file_path (str): Path to the NIfTI file.
    
    Returns:
    List of unique values in the image (which could represent different masks or regions).
    """
    
    # Load the NIfTI file using nibabel
    img = nib.load(nii_file_path)
    
    # Convert the image data to a numpy array
    img_data = img.get_fdata()
    
    # Find the unique values in the image
    unique_values = np.unique(img_data)
    
    return unique_values


path = "/home/local/ASURITE/longchao/Desktop/project/GE_health/CLIP-Driven-Universal-Model/data_temp/simulate/npy/prediction/CT_Abd_FLARE22_Tr_0001_img/CT_Abd_FLARE22_Tr_0001_img_Left Kidney.nii.gz"

values = analyze_nii_values(path)
values

# the values are all (zero, one)

array([0., 1.])

In [6]:
# value = (label, gt_nii.gz, pred_nii.gz)
gt_path = "/home/local/ASURITE/longchao/Desktop/project/GE_health/CLIP-Driven-Universal-Model/data_temp/simulate/npy/CT_Abd/CT_Abd_FLARE22_Tr_0001_gt.nii.gz"
path_liver = "/home/local/ASURITE/longchao/Desktop/project/GE_health/CLIP-Driven-Universal-Model/data_temp/simulate/npy/prediction/CT_Abd_FLARE22_Tr_0001_img/CT_Abd_FLARE22_Tr_0001_img_Liver.nii.gz"

# target: liver: 1








In [10]:
import nibabel as nib
import numpy as np
from scipy.spatial.distance import directed_hausdorff
from medpy.metric.binary import dc
import matplotlib.pyplot as plt

def load_nifti_file(file_path):
    """
    Load a NIfTI file and return the image data as a numpy array.
    """
    img = nib.load(file_path)
    img_data = img.get_fdata()
    return img_data

def calculate_dice(pred_slice, gt_slice, label_value):
    """
    Calculate the Dice coefficient between the predicted liver segmentation and ground truth for a single slice.
    
    Args:
    pred_slice: Predicted binary mask for the slice (0 for background, 1 for liver).
    gt_slice: Ground truth mask slice, where the liver is represented by label_value.
    label_value: The label value of liver in the ground truth.
    
    Returns:
    Dice coefficient for the slice.
    """
    # Binarize the ground truth slice to isolate the liver (label_value)
    gt_liver = (gt_slice == label_value).astype(np.uint8)
    
    # Dice coefficient using MedPy
    dice_score = dc(pred_slice, gt_liver)
    
    return dice_score

def calculate_nsd(pred_slice, gt_slice, label_value, tolerance=1.0):
    """
    Calculate the Normalized Surface Distance (NSD) between the predicted liver segmentation and ground truth for a single slice.
    
    Args:
    pred_slice: Predicted binary mask for the slice (0 for background, 1 for liver).
    gt_slice: Ground truth mask slice, where the liver is represented by label_value.
    label_value: The label value of liver in the ground truth.
    tolerance: Distance tolerance for NSD calculation (in voxels).
    
    Returns:
    NSD score for the slice.
    """
    # Binarize the ground truth mask to isolate the liver (label_value)
    gt_liver = (gt_slice == label_value).astype(np.uint8)
    
    # Extract the surfaces (edges) of both ground truth and predicted liver
    pred_surface = np.argwhere(np.gradient(pred_slice)[0] != 0)
    gt_surface = np.argwhere(np.gradient(gt_liver)[0] != 0)
    
    if len(gt_surface) == 0 or len(pred_surface) == 0:
        return 0  # Avoid division by zero or empty surface case
    
    # Calculate the Hausdorff distance between surfaces (directed Hausdorff distance)
    hausdorff_distance = max(directed_hausdorff(pred_surface, gt_surface)[0], 
                             directed_hausdorff(gt_surface, pred_surface)[0])
    
    # NSD calculation: number of points within the tolerance distance over total points
    nsd = np.sum(hausdorff_distance <= tolerance) / len(gt_surface)
    
    return nsd

def evaluate_slices(gt_data, pred_data, label_value=2, tolerance=1.0):
    """
    Evaluate Dice and NSD slice-by-slice for liver segmentation.
    
    Args:
    gt_data: Ground truth 3D volume with multiple labels.
    pred_data: Predicted 3D volume with binary mask (0 for background, 1 for liver).
    label_value: The label value of liver in the ground truth.
    tolerance: Distance tolerance for NSD calculation (in voxels).
    
    Returns:
    Dice and NSD scores for each slice.
    """
    # Get the number of slices in the z-dimension (axial slices)
    num_slices = gt_data.shape[2]
    
    dice_scores = []
    nsd_scores = []
    
    for i in range(num_slices):
        gt_slice = gt_data[:, :, i]  # Extract a 2D slice from the ground truth
        pred_slice = pred_data[:, :, i]  # Extract a 2D slice from the prediction
        
        # Calculate Dice and NSD for this slice
        dice = calculate_dice(pred_slice, gt_slice, label_value)
        nsd = calculate_nsd(pred_slice, gt_slice, label_value, tolerance)
        
        dice_scores.append(dice)
        nsd_scores.append(nsd)
    
    return dice_scores, nsd_scores




# Get reeady for 3: [1] gt_path, [2] path_liver, [3] label_value: 1 (liver) for the gt data {since gt data might contains multiple labels}
# check at> https://docs.google.com/document/d/1suXpyRQtxZcl0ovdVzQeHi4T_d1UmEU0brOnmMnyai0/edit

# File paths
gt_path = "/home/local/ASURITE/longchao/Desktop/project/GE_health/CLIP-Driven-Universal-Model/data_temp/simulate/npy/CT_Abd/CT_Abd_FLARE22_Tr_0001_gt.nii.gz"
path_liver = "/home/local/ASURITE/longchao/Desktop/project/GE_health/CLIP-Driven-Universal-Model/data_temp/simulate/npy/prediction/CT_Abd_FLARE22_Tr_0001_img/CT_Abd_FLARE22_Tr_0001_img_Liver.nii.gz"

# Load the ground truth and predicted liver masks
gt_data = load_nifti_file(gt_path)
pred_data = load_nifti_file(path_liver)

# Perform slice-level evaluation
dice_scores, nsd_scores = evaluate_slices(gt_data, pred_data, label_value=1, tolerance=1.0)

print("dice_scores_mean:")
print(np.mean(dice_scores))


print("nsd_scores_mean:")
print(np.mean(nsd_scores))


# # Print results for all slices
# for i, (dice, nsd) in enumerate(zip(dice_scores, nsd_scores)):
#     print(f"Slice {i}: Dice = {dice:.4f}, NSD = {nsd:.4f}")

# # Optional: plot the Dice and NSD values for each slice
# plt.figure(figsize=(10, 5))
# plt.plot(dice_scores, label='Dice Coefficient')
# plt.plot(nsd_scores, label='NSD')
# plt.xlabel('Slice Index')
# plt.ylabel('Score')
# plt.title('Slice-by-Slice Evaluation of Liver Segmentation')
# plt.legend()
# plt.show()


dice_scores_mean:
0.6546611706005276
nsd_scores_mean:
0.0
